In [6]:
import warnings; warnings.simplefilter('ignore')
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import pandas as pd
import unidecode, re, textblob
from textblob import TextBlob
from textblob import Word
from tqdm import tqdm

"Done"

'Done'

In [5]:
def gpuTest(verbose=False):
    if torch.cuda.is_available():       
        device = torch.device("cuda")
        if verbose:
            print('There are %d GPU(s) available.' % torch.cuda.device_count())
            print('We will use the GPU:', torch.cuda.get_device_name(0))
    else:
        if verbose: 
            print('No GPU available, using the CPU instead.')
            device = torch.device("cpu")
    return device

def loadBertSentimen():
    pretrained= "mdhugol/indonesia-bert-sentiment-classification"
    model = AutoModelForSequenceClassification.from_pretrained(pretrained)
    tokenizer = AutoTokenizer.from_pretrained(pretrained)    
    return tokenizer, model

def sentiment(text, tokenizer, model, device="cpu"):
    sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    label_index = {'LABEL_0': 1, 'LABEL_1': 0, 'LABEL_2': -1}
    result = sentiment_analysis(text)
    status = label_index[result[0]['label']]
    score = result[0]['score']
    return status, score

def txtPreprocessing(txt, stop=True, lemma=True):
    T = txt.lower() 
    T = re.sub(r'[^\w]',' ',T) # remove symbols
    T = TextBlob(T).words # Tokenisasi, T sekarang List
    if lemma:
        T = [Word(t).lemmatize() for t in T] # Lemma
    if stop:
        T = [t for t in T if str(t) not in stopWords] # Untuk visualisasi data
    return ' '.join(T)

"Done"

'Done'

In [10]:
device_ = gpuTest(verbose=True)
tokenizerS, modelS = loadBertSentimen()
txt = "orang makassar tidak suka makan pisang goreng pakai sambal."
sent, prob = sentiment(txt, tokenizerS, modelS, device=device_)
if sent>0.0:
    sent="Positif"
elif sent<0.0:
    sent="Negatif"
else:
    sent="Netral"
print("Sentiment = {}, Probabilitas = {}".format(sent, prob))

No GPU available, using the CPU instead.


Device set to use cpu


Sentiment = Negatif, Probabilitas = 0.9889833331108093


In [11]:
# Load Data

fData = 'data/Data_Halal.csv'
df = pd.read_csv(fData)
df['sentimen'] = ''
df.head()

,title,href,body,source,sentimen
0,LPPOM MUI | Bogor - Facebook,https://www.facebook.com/halalindonesia/,"LPPOM MUI, Bogor, Indonesia. 145,648 likes · 4...",facebook.com,
1,LPH Halal Nusantara - Facebook,https://www.facebook.com/halalnusantara.id/,"pentingnya sertifikasi halal, tidak hanya dari...",facebook.com,
2,Info Sertifikasi Halal Indonesia - Facebook,https://www.facebook.com/groups/1127431861260327/,Group ini adalah media silaturahmi antar Pelak...,facebook.com,
3,Halal Corridor | Jakarta - Facebook,https://www.facebook.com/profile.php/?id=61565...,"Jun 1, 2025· 󰟠 Seru-seruan bareng Halal Corrid...",facebook.com,
4,PERESMIAN ESQ HALAL CENTER ESQ mendukung progr...,https://www.facebook.com/AryGinanjarAgustian/v...,ESQ mendukung program Halal Indonesia. Selain ...,facebook.com,


In [16]:
for i, d in tqdm(df.iterrows()):
    post_bersih = txtPreprocessing(d.title + ' ' + d.body, stop=False)
    sent, prob = sentiment(post_bersih, tokenizerS, modelS, device=device_)
    if sent>0.0:
        sent="Positif"
    elif sent<0.0:
        sent="Negatif"
    else:
        sent="Netral"
    df.loc[i, 'sentimen'] = sent

0it [00:00, ?it/s]Device set to use cpu
Device set to use cpu
2it [00:00, 14.81it/s]Device set to use cpu
Device set to use cpu
4it [00:00, 15.66it/s]Device set to use cpu
Device set to use cpu
6it [00:00, 17.19it/s]Device set to use cpu
Device set to use cpu
8it [00:00, 16.51it/s]Device set to use cpu
Device set to use cpu
Device set to use cpu
11it [00:00, 17.50it/s]Device set to use cpu
Device set to use cpu
Device set to use cpu
14it [00:00, 17.80it/s]Device set to use cpu
Device set to use cpu
16it [00:00, 17.38it/s]Device set to use cpu
Device set to use cpu
18it [00:01, 17.79it/s]Device set to use cpu
Device set to use cpu
20it [00:01, 17.41it/s]Device set to use cpu
Device set to use cpu
22it [00:01, 16.91it/s]Device set to use cpu
Device set to use cpu
24it [00:01, 17.07it/s]Device set to use cpu
Device set to use cpu
26it [00:01, 17.26it/s]Device set to use cpu
Device set to use cpu
28it [00:01, 16.89it/s]Device set to use cpu
Device set to use cpu
30it [00:01, 17.60it/s]Devi

In [18]:
df.head()

,title,href,body,source,sentimen
0,LPPOM MUI | Bogor - Facebook,https://www.facebook.com/halalindonesia/,"LPPOM MUI, Bogor, Indonesia. 145,648 likes · 4...",facebook.com,Netral
1,LPH Halal Nusantara - Facebook,https://www.facebook.com/halalnusantara.id/,"pentingnya sertifikasi halal, tidak hanya dari...",facebook.com,Positif
2,Info Sertifikasi Halal Indonesia - Facebook,https://www.facebook.com/groups/1127431861260327/,Group ini adalah media silaturahmi antar Pelak...,facebook.com,Netral
3,Halal Corridor | Jakarta - Facebook,https://www.facebook.com/profile.php/?id=61565...,"Jun 1, 2025· 󰟠 Seru-seruan bareng Halal Corrid...",facebook.com,Positif
4,PERESMIAN ESQ HALAL CENTER ESQ mendukung progr...,https://www.facebook.com/AryGinanjarAgustian/v...,ESQ mendukung program Halal Indonesia. Selain ...,facebook.com,Netral


In [19]:
df.to_excel("output-sentimen-halal.xlsx") 